## 目標：將實價登錄的成交檔案，轉換為欲使用的格式。
### 載入套件

In [3]:
import pandas as pd
import numpy as np
import glob
import openpyxl

pd.set_option('display.max_columns', None)

### 匯入資料，直接判斷資料名稱加上縣市

In [4]:
path = '/Users/eason/Documents/to_github/real_estate_data_cleaning/data/input'
fnames = glob.glob(path+r'/*/*_lvr_land_A.csv',recursive=True)
df = pd.DataFrame()
for fn in fnames:
    try:
        lst = pd.read_csv(fn, sep=',',encoding='utf8')

        if fn.split("//")[-1].split("_")[0] == 'F':
            lst['縣市']='新北市'
        elif fn.split("//")[-1].split("_")[0] == 'A':
            lst['縣市']='臺北市'
        elif fn.split("//")[-1].split("_")[0] == 'C':
            lst['縣市']='基隆市'
        elif fn.split("//")[-1].split("_")[0] == 'G':
            lst['縣市']='宜蘭縣'
        elif fn.split("//")[-1].split("_")[0] == 'U':
            lst['縣市']='花蓮縣'
        elif fn.split("//")[-1].split("_")[0] == 'W':
            lst['縣市']='金門縣'
        else : lst['縣市']='其他'

        df = pd.concat([df,lst])

    except:
        print(f'{fn}是空數據集')

/Users/eason/Documents/to_github/data/input/101Q1/F_lvr_land_A.csv是空數據集
/Users/eason/Documents/to_github/data/input/101Q1/A_lvr_land_A.csv是空數據集


/var/folders/cb/j6myjr_n47d2t0x2wc2y44_c0000gn/T/ipykernel_75601/593616047.py:6: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25,28,29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  lst = pd.read_csv(fn, sep=',',encoding='utf8')
/var/folders/cb/j6myjr_n47d2t0x2wc2y44_c0000gn/T/ipykernel_75601/593616047.py:6: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25,28,29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  lst = pd.read_csv(fn, sep=',',encoding='utf8')
/var/folders/cb/j6myjr_n47d2t0x2wc2y44_c0000gn/T/ipykernel_75601/593616047.py:6: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  lst = pd.read_csv(fn, sep=',',encoding='utf8')
/var/folders/cb/j6myjr_n47d2t0x2wc2y44_c0000gn/T/ipykernel_75601/593616047.py:6: DtypeWarning: Columns (3,7,14,15,16,17,18,21,22,24,25) have mixed types. Specify dtype option on import or set low_mem

In [5]:
df.columns

Index(['鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區', '非都市土地使用分區',
       '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態', '主要用途', '主要建材',
       '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛',
       '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別', '車位移轉總面積平方公尺', '車位總價元',
       '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯', '移轉編號', '縣市',
       '車位移轉總面積(平方公尺)'],
      dtype='object')

In [6]:
df = df[['縣市','鄉鎮市區', '交易標的', '土地位置建物門牌', '交易年月日', '移轉層次', '總樓層數', '建物型態','建築完成年月',
         '建物移轉總面積平方公尺', '建物現況格局-房','總價元','單價元平方公尺','備註']]

In [7]:
df_eng_lable_idx = df[df['鄉鎮市區']=='The villages and towns urban district'].index
df = df.drop(df_eng_lable_idx)

In [8]:
# 轉數字 加上19110000
df['交易年月日'] = pd.to_numeric(df['交易年月日'], errors='coerce')
df['交易年月日'] = (df['交易年月日'])+19110000

#先存一個時間格式的交易年
df['交易年'] = pd.to_datetime(df['交易年月日'],format='%Y%m%d',errors='coerce')

# 轉文字 取前6
df['交易年月日'] = df['交易年月日'].astype(str).str.slice(stop=6)

# 再轉為數字
df['交易年月日'] = pd.to_numeric(df['交易年月日'], errors='coerce')

### 季 用一個def判斷Q1~Q4

In [9]:
def season(x):
    if x=='01' or x=='02' or x=='03':
        return 'Q1'
    elif x=='04' or x=='05' or x=='06':
        return 'Q2'
    elif x=='07' or x=='08' or x=='09':
        return 'Q3'
    else:
        return 'Q4'
    
df['季'] = df['交易年月日'].astype(str).str.slice(4,6).apply(season)
df['季'] = df['交易年月日'].astype(str).str.slice(stop=4)+df['季']
df['季']

1       2020Q2
2       2020Q2
3       2020Q2
4       2020Q2
5       2020Q2
         ...  
6538    2021Q1
6539    2021Q1
6540    2021Q1
6541    2020Q4
6542    2018Q1
Name: 季, Length: 181729, dtype: object

### 建築完成年月 也改成西元

In [10]:
# 轉數字 加上19110000
df['建築完成年月'] = pd.to_numeric(df['建築完成年月'], errors='coerce')
df['建築完成年月'] = (df['建築完成年月'])+19110000


df['建築完成年'] = pd.to_datetime(df['建築完成年月'],format='%Y%m%d',errors='coerce')


# 轉文字 取前6
df['建築完成年月'] = df['建築完成年月'].astype(str).str.slice(stop=6)

# 再轉為數字
df['建築完成年月'] = df['建築完成年月'].astype("int",errors='ignore')
df['建築完成年月']

1       201202
2          nan
3          nan
4       198101
5          nan
         ...  
6538    199802
6539       nan
6540       nan
6541       nan
6542    202007
Name: 建築完成年月, Length: 181729, dtype: object

In [11]:
# df['交易年'] = df['交易年月日'].astype(str).str.slice(stop=4).astype("float", errors='ignore')
# df['建築完成年'] = df['建築完成年月'].astype(str).str.slice(stop=4).astype("float", errors='ignore')
# df['屋齡']  = (df['交易年']-df['建築完成年'])
df['屋齡'] =((df['交易年']-df['建築完成年']).dt.days/365).round(1)
df['屋齡']

1        8.2
2        NaN
3        NaN
4       39.2
5        NaN
        ... 
6538    23.0
6539     NaN
6540     NaN
6541     NaN
6542    -2.3
Name: 屋齡, Length: 181729, dtype: float64

### 地址切割路 段

In [12]:
df['土地位置建物門牌'] = df['土地位置建物門牌'].str.extract(r'(.*[區鄉市鎮])?(.*)')[1]

In [13]:
df['路'] = df['土地位置建物門牌'].str.extract(r'(.*?[路街道段弄村坑乾寮堡]+)?(.*)')[0]
df['其他'] = df['土地位置建物門牌'].str.extract(r'(.*?[路街道段弄村坑乾寮堡]+)?(.*)')[1]
df['段'] = df['其他'].str.extract(r'(.*[段])?(.*)')[0]
df['其他'] = df['其他'].str.extract(r'(.*[段])?(.*)')[1]

In [14]:
def house_year_interval(x):
    if x<10:
        return '0-10年'
    elif x<20:
        return '10-20年'
    elif x<30:
        return '20-30年'
    elif x<40:
        return '30-40年'
    elif x<50:
        return '40-50年'
    elif x<60:
        return '50-60年'
    else:
        return '60年以上'

df['屋齡區間'] = df['屋齡'].apply(house_year_interval)

In [15]:
df['建物移轉總面積(坪)'] = df['建物移轉總面積平方公尺'].astype('float')*0.3025

In [16]:
def land_interval(x):
    if x<10:
        return '0-10坪'
    elif x<20:
        return '10-20坪'
    elif x<30:
        return '20-30坪'
    elif x<40:
        return '30-40坪'
    elif x<50:
        return '50-60坪'
    else:
        return '60坪以上'

df['面積區間'] = df['建物移轉總面積(坪)'].apply(land_interval)

In [17]:
df['總價元(萬)'] = df['總價元'].astype('float')/10000
df['單價(萬/坪)'] = df['總價元(萬)']/df['建物移轉總面積(坪)']

In [18]:
df

,縣市,鄉鎮市區,交易標的,土地位置建物門牌,交易年月日,移轉層次,總樓層數,建物型態,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,總價元,單價元平方公尺,備註,交易年,季,建築完成年,屋齡,路,其他,段,屋齡區間,建物移轉總面積(坪),面積區間,總價元(萬),單價(萬/坪)
1,其他,新店區,房地(土地+建物),北新路一段２之１９號六樓,202005,六層,十三層,住宅大樓(11層含以上有電梯),201202,75.35,2,12720000,168812,NaN,2020-05-02,2020Q2,2012-02-24,8.2,北新路,２之１９號六樓,一段,0-10年,22.793375,20-30坪,1272.0,55.805689
2,其他,深坑區,土地,永安段388-2地號,202004,NaN,NaN,其他,nan,0.0,0,29336000,87722,NaN,2020-04-01,2020Q2,NaT,NaN,永安段,388-2地號,NaN,60年以上,0.000000,0-10坪,2933.6,inf
3,其他,新店區,土地,錦秀段749地號,202004,NaN,NaN,其他,nan,0.0,0,1300000,16297,向政府機關承購之案件。；,2020-04-08,2020Q2,NaT,NaN,錦秀段,749地號,NaN,60年以上,0.000000,0-10坪,130.0,inf
4,其他,土城區,房地(土地+建物),明德路一段３１３巷５弄３號五樓,202004,五層，陽台,五層,公寓(5樓含以下無電梯),198101,65.16,3,3700000,56783,含增建或未登記建物。；受債權債務影響或債務抵償之交易。；,2020-04-06,2020Q2,1981-01-16,39.2,明德路,３１３巷５弄３號五樓,一段,30-40年,19.710900,10-20坪,370.0,18.771340
5,其他,汐止區,土地,同新段342地號,202004,NaN,NaN,其他,nan,0.0,0,1016000,7417,標售地籍清理。,2020-04-08,2020Q2,NaT,NaN,同新段,342地號,NaN,60年以上,0.000000,0-10坪,101.6,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538,其他,文山區,房地(土地+建物),萬美街二段８４號六樓之２,202101,六層,十層,華廈(10層含以下有電梯),199802,79.24,2,14000000,176678,其他增建；,2021-01-26,2021Q1,1998-02-13,23.0,萬美街,８４號六樓之２,二段,20-30年,23.970100,20-30坪,1400.0,58.406098
6539,其他,文山區,房地(土地+建物),興隆路一段２７１巷４弄４號二樓,202101,二層，陽台,四層,公寓(5樓含以下無電梯),nan,101.28,3,14520000,143365,陽台外推；其他增建；,2021-01-21,2021Q1,NaT,NaN,興隆路,２７１巷４弄４號二樓,一段,60年以上,30.637200,30-40坪,1452.0,47.393365
6540,其他,文山區,房地(土地+建物),景華街８２巷１號四樓,202101,四層,四層,公寓(5樓含以下無電梯),nan,107.07,4,15500000,144765,陽台外推；其他增建；,2021-01-27,2021Q1,NaT,NaN,景華街,８２巷１號四樓,NaN,60年以上,32.388675,30-40坪,1550.0,47.856234
6541,其他,中正區,車位,延平南路６７號等共有部分,202012,NaN,NaN,其他,nan,29.04,0,1500000,NaN,僅車位交易；,2020-12-24,2020Q4,NaT,NaN,延平南路,６７號等共有部分,NaN,60年以上,8.784600,0-10坪,150.0,17.075336


In [35]:
df = df[~df['土地位置建物門牌'].str.contains('魚坑')]
df = df[~df['土地位置建物門牌'].str.contains('魚堀')]
df = df[~df['備註'].str.contains('寘').fillna(False)]


In [36]:
df.columns

Index(['縣市', '鄉鎮市區', '交易標的', '土地位置建物門牌', '路', '段', '其他', '交易年月日', '季', '移轉層次',
       '總樓層數', '建物型態', '建築完成年月', '屋齡', '屋齡區間', '建物移轉總面積(坪)', '面積區間',
       '建物現況格局-房', '總價元(萬)', '單價(萬/坪)', '備註'],
      dtype='object')

In [37]:
df=df[['縣市', '鄉鎮市區', '交易標的', '土地位置建物門牌','路','段','其他','交易年月日','季',
    '移轉層次','總樓層數','建物型態','建築完成年月','屋齡','屋齡區間','建物移轉總面積(坪)','面積區間','建物現況格局-房','總價元(萬)','單價(萬/坪)','備註']]

In [38]:
df.to_excel('/Users/eason/Documents/to_github/real_estate_data_cleaning/data/output/成交資料.xlsx',index=False)